-------------------------------------------------------------------------------------------------------------------
# 1. Import library
-------------------------------------------------------------------------------------------------------------------

In [1]:
# Dataframe manipulation
import numpy as np
import pandas as pd
import math

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
# sns.set(style = 'whitegrid',context = 'notebook')

# Preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

# Modelling Helpers:
from sklearn.preprocessing import Normalizer, scale
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, ShuffleSplit, cross_validate
from sklearn import model_selection
from sklearn.model_selection import train_test_split

# Classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
# Evaluation metrics for Classification
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score

# Regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV,ElasticNet,LogisticRegression
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
# Evaluation metrics for Regression 
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error

# Configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

print("Setup complete...")

Setup complete...


-------------------------------------------------------------------------------------------------------------------
# 2. Common function
-------------------------------------------------------------------------------------------------------------------

In [2]:
def Check_Missing_Data(df):    
    # count all missing values of each column
    total = df.isnull().sum().sort_values(ascending=False)
    # calculate percentage of null values for each column
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

-------------------------------------------------------------------------------------------------------------------
# 3. Import dataset
-------------------------------------------------------------------------------------------------------------------

In [111]:
# load dataset
df = pd.read_csv('London_listings.csv')
print(df.shape)
df.head()

(77096, 96)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,9554,https://www.airbnb.com/rooms/9554,20181207034825,2018-12-07,"Cozy, 3 minutes to Piccadilly Line",PLEASE CONTACT ME BEFORE BOOKING Homely apartm...,"Hello people, This is a bright, comfortable ro...",PLEASE CONTACT ME BEFORE BOOKING Homely apartm...,none,Details to follow..,NaN,details to follow when i get a chance..,"Kitchen, small dining / smoking room (if you s...",Depends on what else I have on.. It is always ...,"If you smoke, please do so only in the little ...",NaN,NaN,https://a0.muscache.com/im/pictures/77912984/a...,NaN,31655,https://www.airbnb.com/users/show/31655,Guy,2009-08-14,"London, England, United Kingdom",Please contact me before booking! Please see d...,within a few hours,100%,NaN,t,https://a0.muscache.com/im/pictures/9cae5e2c-3...,https://a0.muscache.com/im/pictures/9cae5e2c-3...,LB of Haringey,4.0,4.0,"['email', 'phone', 'manual_online', 'reviews',...",t,f,"London, United Kingdom",LB of Haringey,Haringey,NaN,London,NaN,N8 0EY,London,"London, United Kingdom",GB,United Kingdom,51.587767,-0.105666,f,Apartment,Private room,2,NaN,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Smoking allowed"",Br...",NaN,$35.00,$198.00,$788.00,NaN,$7.00,1,$15.00,1,730,a week ago,t,18,18,18,291,2018-12-07,133,2012-04-26,2018-08-13,97.0,10.0,9.0,10.0,10.0,9.0,10.0,f,NaN,NaN,f,f,strict_14_with_grace_period,t,f,4,1.65
1,11076,https://www.airbnb.com/rooms/11076,20181207034825,2018-12-07,The Sanctuary,The room has a double bed and a single foldawa...,This Listing is for The Sanctury The accommoda...,The room has a double bed and a single foldawa...,none,"Ealing Broadway, as short walk from our place ...",NaN,extemely good transport links to central londo...,Huge family kitchen and good wifi,We give everyone a great welcome and make sure...,We don't allow alcohol or takeaways,NaN,NaN,https://a0.muscache.com/im/pictures/a0f4c78a-6...,NaN,40471,https://www.airbnb.com/users/show/40471,Rosa,2009-09-22,"London, England, United Kingdom","Hi, I'm Rosa, I'm one of the owners of the At-...",within a day,67%,NaN,f,https://a0.muscache.com/im/users/40471/profile...,https://a0.muscache.com/im/users/40471/profile...,LB of Ealing,6.0,6.0,"['email', 'phone', 'facebook', 'reviews', 'wor...",t,f,"Ealing, Greater London, United Kingdom",LB of Ealing,Ealing,NaN,Ealing,Greater London,W13 8,London,"Ealing, United Kingdom",GB,United Kingdom,51.515645,-0.314508,t,Apartment,Private room,2,NaN,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Breakfast,""Pets l...",NaN,$70.00,NaN,NaN,NaN,NaN,2,$35.00,2,1125,4 months ago,t,0,0,0,0,2018-12-07,

-------------------------------------------------------------------------------------------------------------------
# 4. Fixing incorrect data
-------------------------------------------------------------------------------------------------------------------

### remove redundant features

In [112]:
remove_list = ['listing_url', 'scrape_id', 'last_scraped', 'name',       
              'interaction', 'house_rules', 'thumbnail_url', 'medium_url',
              'picture_url', 'xl_picture_url', 'host_id', 'host_url',
              'host_thumbnail_url', 'host_picture_url',       
              'host_has_profile_pic',
              'calendar_last_scraped', 
              'first_review', 'last_review',
              'requires_license', 'license', 'jurisdiction_names',
              'is_business_travel_ready',
              'require_guest_profile_picture',
              'require_guest_phone_verification']
print(len(remove_list))        # 25
selected_features = [e for e in df.columns.values if e not in remove_list]
len(selected_features)   # 70
df = df.loc[:,selected_features]
df.shape

24


(77096, 72)

### Converting columns from string to float

In [113]:
df['price'] = (df['price'].str.replace(r'[^-+\d.]','').astype(float))
df['extra_people'] = (df['extra_people'].str.replace(r'[^-+\d.]','').astype(float))
df['cleaning_fee'] = (df['cleaning_fee'].str.replace(r'[^-+\d.]','').astype(float))

### Remove outliers (price > 1000)

The incorrect data is found during exploration stage

In [114]:
df = df.loc[df.price <= 1000,:]
df.shape

(76889, 72)

In [115]:
df.country_code.value_counts()

GB    76888
ES        1
Name: country_code, dtype: int64

In [116]:
df = df.loc[df.country_code=='GB',:]
df.shape

(76888, 72)

### remove rows with target variable as 0

In [117]:
print(df[df.price == 0].shape)
print(df[df.price > 0].shape)
# remove rows with price ==0
df = df[df.price > 0]
df = df.reset_index(drop = True)
df.shape

(33, 72)
(76855, 72)


(76855, 72)

-------------------------------------------------------------------------------------------------------------------
# 5. Construct Input & Ouput variables
-------------------------------------------------------------------------------------------------------------------

### Selected vars for independent vars

In [118]:
df.head()

,id,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count,reviews_per_month
0,9554,PLEASE CONTACT ME BEFORE BOOKING Homely apartm...,"Hello people, This is a bright, comfortable ro...",PLEASE CONTACT ME BEFORE BOOKING Homely apartm...,none,Details to follow..,NaN,details to follow when i get a chance..,"Kitchen, small dining / smoking room (if you s...",Guy,2009-08-14,"London, England, United Kingdom",Please contact me before booking! Please see d...,within a few hours,100%,NaN,t,LB of Haringey,4.0,4.0,"['email', 'phone', 'manual_online', 'reviews',...",f,"London, United Kingdom",LB of Haringey,Haringey,NaN,London,NaN,N8 0EY,London,"London, United Kingdom",GB,United Kingdom,51.587767,-0.105666,f,Apartment,Private room,2,NaN,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Smoking allowed"",Br...",NaN,35.0,$198.00,$788.00,NaN,7.0,1,15.0,1,730,a week ago,t,18,18,18,291,133,97.0,10.0,9.0,10.0,10.0,9.0,10.0,f,strict_14_with_grace_period,4,1.65
1,11076,The room has a double bed and a single foldawa...,This Listing is for The Sanctury The accommoda...,The room has a double bed and a single foldawa...,none,"Ealing Broadway, as short walk from our place ...",NaN,extemely good transport links to central londo...,Huge family kitchen and good wifi,Rosa,2009-09-22,"London, England, United Kingdom","Hi, I'm Rosa, I'm one of the owners of the At-...",within a day,67%,NaN,f,LB of Ealing,6.0,6.0,"['email', 'phone', 'facebook', 'reviews', 'wor...",f,"Ealing, Greater London, United Kingdom",LB of Ealing,Ealing,NaN,Ealing,Greater London,W13 8,London,"Ealing, United Kingdom",GB,United Kingdom,51.515645,-0.314508,t,Apartment,Private room,2,NaN,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Breakfast,""Pets l...",NaN,70.0,NaN,NaN,NaN,NaN,2,35.0,2,1125,4 months ago,t,0,0,0,0,3,90.0,8.0,9.0,10.0,9.0,9.0,9.0,t,strict_14_with_grace_period,6,0.09
2,13913,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,For art lovers I can give guest my Tate Member...,The flat only a 10 minute walk to Finsbury Par...,Guest will have access to the self catering ki...,Alina,2009-11-16,"London, England, United Kingdom",I am a Multi-Media Visual Artist and Creative ...,within a day,67%,NaN,f,LB of Islington,4.0,4.0,"['email', 'phone', 'facebook', 'reviews']",f,"Islington, Greater London, United Kingdom",LB of Islington,Islington,NaN,Islington,Greater London,N4 3,London,"Islington, United Kingdom",GB,United Kingdom,51.568017,-0.111208,t,Apartment,Private room,2,1.0,1.0,0.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,""Paid parking off ...",538.0,65.0,$333.00,"$1,176.00",$100.00,15.0,1,15.0,1,29,2 weeks ago,t,29,59,89,364,14,95.0,9.0,10.0,9.0,10.0,9.0,9.0,f,moderate,3,0.14
3,17402,"Open from June 2018 after a 3-year break, we a...",Ready again from June 2018 for bookings after ...,"Open from June 2018 after a 3-year break, we a...",none,"Loca

In [122]:
cat_vars = ['room_type','cancellation_policy','host_is_superhost','neighbourhood_cleansed']
print('Number of categorical variables: ', len(cat_vars))
# let's visualise the values of the categorical variables
df[cat_vars].head(3)

Number of categorical variables:  4


,room_type,cancellation_policy,host_is_superhost,neighbourhood_cleansed
0,Private room,strict_14_with_grace_period,t,Haringey
1,Private room,strict_14_with_grace_period,f,Ealing
2,Private room,moderate,f,Islington


In [123]:
# List of numerical variables
num_vars = [var for var in df.columns if df[var].dtypes != 'O' and var not in ['price']]
print('Number of numerical variables', len(num_vars))
X = df[num_vars + cat_vars]
print(X.shape)
X.head()

Number of numerical variables 31
(76855, 35)


,id,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,room_type,cancellation_policy,host_is_superhost,neighbourhood_cleansed
0,9554,NaN,4.0,4.0,NaN,51.587767,-0.105666,2,NaN,1.0,1.0,NaN,7.0,1,15.0,1,730,18,18,18,291,133,97.0,10.0,9.0,10.0,10.0,9.0,10.0,4,1.65,Private room,strict_14_with_grace_period,t,Haringey
1,11076,NaN,6.0,6.0,NaN,51.515645,-0.314508,2,NaN,1.0,1.0,NaN,NaN,2,35.0,2,1125,0,0,0,0,3,90.0,8.0,9.0,10.0,9.0,9.0,9.0,6,0.09,Private room,strict_14_with_grace_period,f,Ealing
2,13913,NaN,4.0,4.0,NaN,51.568017,-0.111208,2,1.0,1.0,0.0,538.0,15.0,1,15.0,1,29,29,59,89,364,14,95.0,9.0,10.0,9.0,10.0,9.0,9.0,3,0.14,Private room,moderate,f,Islington
3,17402,NaN,15.0,15.0,NaN,51.520982,-0.140024,6,2.0,3.0,3.0,NaN,65.0,4,10.0,3,365,26,56,86,360,39,93.0,10.0,9.0,9.0,9.0,10.0,9.0,13,0.42,Entire home/apt,strict_14_with_grace_period,t,Westminster
4,24328,NaN,2.0,2.0,NaN,51.472981,-0.163764,4,1.5,2.0,2.0,1001.0,70.0,2,15.0,90,90,28,58,88,363,92,98.0,10.0,10.0,10.0,10.0,9.0,9.0,1,0.94,Entire home/apt,strict_14_with_grace_period,f,Wandsworth


### Selected vars for dependent vars

In [124]:
## df['log_price'] = np.log(df.price)
## df['log_price'][0:3]
Y = df['price']

In [125]:
df = pd.concat([X,Y],1)
print(df.shape)
df.head(3)

(76855, 36)


,id,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,room_type,cancellation_policy,host_is_superhost,neighbourhood_cleansed,price
0,9554,NaN,4.0,4.0,NaN,51.587767,-0.105666,2,NaN,1.0,1.0,NaN,7.0,1,15.0,1,730,18,18,18,291,133,97.0,10.0,9.0,10.0,10.0,9.0,10.0,4,1.65,Private room,strict_14_with_grace_period,t,Haringey,35.0
1,11076,NaN,6.0,6.0,NaN,51.515645,-0.314508,2,NaN,1.0,1.0,NaN,NaN,2,35.0,2,1125,0,0,0,0,3,90.0,8.0,9.0,10.0,9.0,9.0,9.0,6,0.09,Private room,strict_14_with_grace_period,f,Ealing,70.0
2,13913,NaN,4.0,4.0,NaN,51.568017,-0.111208,2,1.0,1.0,0.0,538.0,15.0,1,15.0,1,29,29,59,89,364,14,95.0,9.0,10.0,9.0,10.0,9.0,9.0,3,0.14,Private room,moderate,f,Islington,65.0


-------------------------------------------------------------------------------------------------------------------
# 6. Separate dataset into train and test
-------------------------------------------------------------------------------------------------------------------

This is to avoid over-fitting. This step involves randomness, therefore, it's needed to set the seed.

In [126]:
# Let's separate into train and test set
# Remember to set the seed (random_state for this sklearn function)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.1,random_state=0) # we are setting the seed here
X_train.shape, X_test.shape

((69169, 35), (7686, 35))

In [127]:
X_train.head()

,id,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,room_type,cancellation_policy,host_is_superhost,neighbourhood_cleansed
6065,4063841,NaN,1.0,1.0,NaN,51.468400,-0.290640,8,2.0,4.0,4.0,NaN,35.0,1,0.0,7,1125,12,12,12,12,2,80.0,8.0,5.0,7.0,8.0,7.0,7.0,1,0.06,Entire home/apt,strict_14_with_grace_period,f,Richmond upon Thames
40338,20231469,NaN,2.0,2.0,NaN,51.574398,-0.072689,3,1.0,2.0,2.0,NaN,40.0,2,15.0,92,366,0,0,0,0,5,100.0,10.0,10.0,10.0,10.0,9.0,9.0,1,0.32,Entire home/apt,strict_14_with_grace_period,f,Hackney
52174,23712985,NaN,1.0,1.0,NaN,51.547731,-0.149095,1,1.0,0.0,1.0,NaN,NaN,1,0.0,7,1125,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.16,Private room,flexible,f,Camden
2377,1080216,NaN,2.0,2.0,NaN,51.427245,-0.127745,2,1.5,1.0,1.0,NaN,NaN,1,10.0,3,90,0,4,34,34,12,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2,0.40,Private room,moderate,t,Lambeth
8479,5703142,NaN,1.0,1.0,NaN,51.444074,-0.158069,7,2.5,4.0,7.0,NaN,NaN,1,0.0,5,1125,2,3,3,3,9,97.0,10.0,10.0,10.0,10.0,9.0,10.0,1,0.22,Entire home/apt,strict_14_with_grace_period,f,Wandsworth


----------------------------------------------------------------------------------------------------------------------

# 7. Handling missing values

### 2.1. Categorical variables

For categorical variables, fill missing information by adding an additional category: "missing"

In [128]:
# make a list of the categorical variables that contain missing values
categorical_list = [var for var in df.columns if df[var].dtypes=='O']
missing_df = Check_Missing_Data(df[categorical_list])
missing_df.head()

,Total,Percent
host_is_superhost,7,0.000091
neighbourhood_cleansed,0,0.000000
cancellation_policy,0,0.000000
room_type,0,0.000000


In [129]:
missing_df = missing_df.loc[missing_df.Percent < 0.3,]
missing_df.shape
categorical_list = missing_df.index.values
len(categorical_list)

4

In [130]:
# function to replace NA in categorical variables
def fill_categorical_na(df, var_list):
    X = df.copy()
    X[var_list] = df[var_list].fillna('Missing')
    return X
                                      
# replace missing values with new label: "Missing"
X_train = fill_categorical_na(X_train, categorical_list)
# check that we have no missing information in the engineered variables
[var for var in missing_df.index.values if X_train[var].isnull().sum()>0]

[]

In [131]:
# replace missing values with new label: "Missing"
X_test = fill_categorical_na(X_test, missing_df.index.values)
# check that we have no missing information in the engineered variables
[var for var in missing_df.index.values if X_test[var].isnull().sum()>0]

[]

### 2.2. Numerical variables

For numerical variables, add an additional variable to capture the missing information, and then replace the missing information in the original variable by the mode, or most frequent value:

In [132]:
removed_list = ['id','price']
# make a list of the categorical variables that contain missing values
numerical_list = [var for var in df.columns if df[var].dtypes!='O' and var not in removed_list]

# alternative approach
# numerics= ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
# numdf = df.select_dtypes(include=numerics)
# numerical_list = numdf.columns
# numerical_list

missing_df = Check_Missing_Data(df[numerical_list])
missing_df.shape

(30, 2)

In [133]:
missing_df = missing_df.loc[missing_df.Percent < 0.3,]
missing_df.shape
numerical_list = missing_df.index.values
len(numerical_list)

27

In [134]:
# function to replace NA in categorical variables
def fill_numerical_na(df, var_list):
    X = df.copy()
    for var in var_list:
        # calculate the mode
        mode_val = X[var].mode()[0]
        X[var].fillna(mode_val, inplace=True)
    return X

In [135]:
# replace mode the missing values
X_train = fill_numerical_na(X_train, numerical_list)
# check that we have no missing information in the engineered variables
[var for var in missing_df.index.values if X_train[var].isnull().sum()>0]

[]

In [136]:
# replace mode the missing values
X_test = fill_numerical_na(X_test, missing_df.index.values)
# check that we have no missing information in the engineered variables
[var for var in missing_df.index.values if X_test[var].isnull().sum()>0]


[]

In [137]:
print(X_train.shape)
print(X_test.shape)

(69169, 35)
(7686, 35)


----------------------------------------------------------------------------------------------------------------------

# 8. Data Transformation 

## 8.1 Numerical variables

### Log Transform Non-Gaussian distributed variables

We will log transform the numerical variables that do not contain zeros in order to get a more Gaussian-like distribution. This tends to help Linear machine learning models. 

In [138]:
# df['log_price'] = np.log(df['price'])
# df['log_price'] = np.log(df['price'])

### Feature Scaling

In [139]:
# fit scaler
scaler = MinMaxScaler() # create an instance
scaler.fit(X_train[numerical_list]) #  fit  the scaler to the train set for later use

# transform the train and test set, and add on the Id and SalePrice variables
train_num_df = pd.DataFrame(scaler.transform(X_train[numerical_list]), columns=numerical_list)
test_num_df = pd.DataFrame(scaler.transform(X_test[numerical_list]), columns=numerical_list)

In [140]:
train_num_df.head()

,cleaning_fee,review_scores_value,review_scores_location,review_scores_checkin,review_scores_accuracy,review_scores_communication,review_scores_cleanliness,review_scores_rating,reviews_per_month,bathrooms,beds,bedrooms,host_listings_count,host_total_listings_count,number_of_reviews,latitude,longitude,accommodates,availability_60,availability_30,availability_365,availability_90,guests_included,extra_people,calculated_host_listings_count,maximum_nights,minimum_nights
0,0.057003,0.625,0.625,0.625,0.75,0.75,0.375,0.7500,0.003021,0.181818,0.20,0.090909,0.000674,0.000674,0.003781,0.445075,0.267782,0.466667,0.200000,0.400000,0.032877,0.133333,0.000000,0.000000,0.000000,5.234033e-07,0.005338
1,0.065147,0.875,0.875,1.000,1.00,1.00,1.000,1.0000,0.018731,0.090909,0.10,0.045455,0.001348,0.001348,0.009452,0.720847,0.544828,0.133333,0.000000,0.000000,0.000000,0.000000,0.043478,0.063830,0.000000,1.699664e-07,0.080961
2,0.032573,1.000,1.000,1.000,1.00,1.00,1.000,1.0000,0.009063,0.090909,0.05,0.000000,0.000674,0.000674,0.001890,0.651469,0.447706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.234033e-07,0.005338
3,0.032573,1.000,1.000,1.000,1.00,1.00,1.000,1.0000,0.023565,0.136364,0.05,0.022727,0.001348,0.001348,0.022684,0.338000,0.474844,0.066667,0.066667,0.000000,0.093151,0.377778,0.000000,0.042553,0.000922,4.144385e-08,0.001779
4,0.032573,1.000,0.875,1.000,1.00,1.00,1.000,0.9625,0.012689,0.227273,0.35,0.090909,0.000674,0.000674,0.017013,0.381785,0.436299,0.400000,0.050000,0.066667,0.008219,0.033333,0.000000,0.000000,0.000000,5.234033e-07,0.003559


In [141]:
print(train_num_df.shape)
print(test_num_df.shape)

(69169, 27)
(7686, 27)


## 8.2. Categorical variables

In [97]:
df.room_type.value_counts()

Entire home/apt    42553
Private room       33559
Shared room          743
Name: room_type, dtype: int64

should joint distinct words

In [152]:
def good_location(neigborhood):
    if(neigborhood in ['City of London','Kensington and Chelsea','Westminster',
                        'Camden','ammersmith and Fulham','Richmond upon Thames',
                        'Islington']):
        return 1
    else: 
        return 0

X_train['good_nbh'] = X_train['neighbourhood_cleansed'].apply(good_location)
X_test['good_nbh'] = X_test['neighbourhood_cleansed'].apply(good_location)

In [153]:
categorical_list = ['host_is_superhost','cancellation_policy', 'room_type','good_nbh']

In [155]:
cat_df = pd.DataFrame()
full_ds = pd.concat(objs=[X_train[categorical_list], X_test[categorical_list]], axis=0)
for var in ['host_is_superhost','cancellation_policy', 'room_type']:
    var_dummies = pd.get_dummies(full_ds[var], prefix=var, prefix_sep='_',drop_first=True)
    cat_df = pd.concat([cat_df, var_dummies], axis=1)

cat_df['good_nbh'] = full_ds['good_nbh']
cat_df.columns

Index(['host_is_superhost_f', 'host_is_superhost_t', 'cancellation_policy_moderate', 'cancellation_policy_strict', 'cancellation_policy_strict_14_with_grace_period', 'cancellation_policy_super_strict_30', 'cancellation_policy_super_strict_60', 'room_type_Private room', 'room_type_Shared room', 'good_nbh'], dtype='object')

In [ ]:
cat_df

In [156]:
cat_df.columns = ['host_is_superhost_f', 'host_is_superhost_t', 'cancellation_policy_moderate', 'cancellation_policy_strict', 'cancellation_policy_strict_14_with_grace_period', 'cancellation_policy_super_strict_30', 'cancellation_policy_super_strict_60', 'room_type_Private', 'room_type_Shared','good_nbh']
l = X_train.shape[0]
train_cat_df = cat_df[:l].reset_index(drop=True)
test_cat_df = cat_df[l:].reset_index(drop=True)

In [157]:
train_cat_df.head()

,host_is_superhost_f,host_is_superhost_t,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Private,room_type_Shared,good_nbh
0,1,0,0,0,1,0,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,1
3,0,1,1,0,0,0,0,1,0,0
4,1,0,0,0,1,0,0,0,0,0


In [159]:
[var for var in train_cat_df.columns if train_cat_df[var].isnull().sum()>0]

[]

In [160]:
[var for var in test_cat_df.columns if test_cat_df[var].isnull().sum()>0]

[]

## 8.3. Text vars

## 8.4. Temporal vars

## 8.5. Selected features

In [161]:
selected_features = list(numerical_list) + list(cat_df.columns.values)
print(len(selected_features))
selected_features

37


['cleaning_fee',
 'review_scores_value',
 'review_scores_location',
 'review_scores_checkin',
 'review_scores_accuracy',
 'review_scores_communication',
 'review_scores_cleanliness',
 'review_scores_rating',
 'reviews_per_month',
 'bathrooms',
 'beds',
 'bedrooms',
 'host_listings_count',
 'host_total_listings_count',
 'number_of_reviews',
 'latitude',
 'longitude',
 'accommodates',
 'availability_60',
 'availability_30',
 'availability_365',
 'availability_90',
 'guests_included',
 'extra_people',
 'calculated_host_listings_count',
 'maximum_nights',
 'minimum_nights',
 'host_is_superhost_f',
 'host_is_superhost_t',
 'cancellation_policy_moderate',
 'cancellation_policy_strict',
 'cancellation_policy_strict_14_with_grace_period',
 'cancellation_policy_super_strict_30',
 'cancellation_policy_super_strict_60',
 'room_type_Private',
 'room_type_Shared',
 'good_nbh']

----------------------------------------------------------------------------------------------------------------------

## Save preprocessed dataset

In [162]:
train_ds = pd.concat([pd.DataFrame(X_train[['id']]).reset_index(drop=True),train_num_df,train_cat_df.reset_index(drop=True),pd.DataFrame(y_train).reset_index(drop=True)],axis = 1)
test_ds = pd.concat([pd.DataFrame(X_test[['id']]).reset_index(drop=True),test_num_df,test_cat_df.reset_index(drop=True),pd.DataFrame(y_test).reset_index(drop=True)],axis = 1)
print(train_ds.shape)
print(test_ds.shape)

(69169, 39)
(7686, 39)


In [163]:
train_ds.head()

,id,cleaning_fee,review_scores_value,review_scores_location,review_scores_checkin,review_scores_accuracy,review_scores_communication,review_scores_cleanliness,review_scores_rating,reviews_per_month,bathrooms,beds,bedrooms,host_listings_count,host_total_listings_count,number_of_reviews,latitude,longitude,accommodates,availability_60,availability_30,availability_365,availability_90,guests_included,extra_people,calculated_host_listings_count,maximum_nights,minimum_nights,host_is_superhost_f,host_is_superhost_t,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Private,room_type_Shared,good_nbh,price
0,4063841,0.057003,0.625,0.625,0.625,0.75,0.75,0.375,0.7500,0.003021,0.181818,0.20,0.090909,0.000674,0.000674,0.003781,0.445075,0.267782,0.466667,0.200000,0.400000,0.032877,0.133333,0.000000,0.000000,0.000000,5.234033e-07,0.005338,1,0,0,0,1,0,0,0,0,1,160.0
1,20231469,0.065147,0.875,0.875,1.000,1.00,1.00,1.000,1.0000,0.018731,0.090909,0.10,0.045455,0.001348,0.001348,0.009452,0.720847,0.544828,0.133333,0.000000,0.000000,0.000000,0.000000,0.043478,0.063830,0.000000,1.699664e-07,0.080961,1,0,0,0,1,0,0,0,0,0,77.0
2,23712985,0.032573,1.000,1.000,1.000,1.00,1.00,1.000,1.0000,0.009063,0.090909,0.05,0.000000,0.000674,0.000674,0.001890,0.651469,0.447706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.234033e-07,0.005338,1,0,0,0,0,0,0,1,0,1,26.0
3,1080216,0.032573,1.000,1.000,1.000,1.00,1.00,1.000,1.0000,0.023565,0.136364,0.05,0.022727,0.001348,0.001348,0.022684,0.338000,0.474844,0.066667,0.066667,0.000000,0.093151,0.377778,0.000000,0.042553,0.000922,4.144385e-08,0.001779,0,1,1,0,0,0,0,1,0,0,25.0
4,5703142,0.032573,1.000,0.875,1.000,1.00,1.00,1.000,0.9625,0.012689,0.227273,0.35,0.090909,0.000674,0.000674,0.017013,0.381785,0.436299,0.400000,0.050000,0.066667,0.008219,0.033333,0.000000,0.000000,0.000000,5.234033e-07,0.003559,1,0,0,0,1,0,0,0,0,0,180.0


In [164]:
train_ds.to_csv('preprocessed_train_exp05.csv',index=False)
test_ds.to_csv('preprocessed_test_exp05.csv',index=False)

In [165]:
# now we save the selected list of features
pd.Series(train_ds.columns).to_csv('selected_features.csv', index=False)

In [166]:
pd.Series(train_ds.columns)

0                                                  id
1                                        cleaning_fee
2                                 review_scores_value
3                              review_scores_location
4                               review_scores_checkin
5                              review_scores_accuracy
6                         review_scores_communication
7                           review_scores_cleanliness
8                                review_scores_rating
9                                   reviews_per_month
10                                          bathrooms
11                                               beds
12                                           bedrooms
13                                host_listings_count
14                          host_total_listings_count
15                                  number_of_reviews
16                                           latitude
17                                          longitude
18                          